In [1]:
import lancedb
import re
import pickle
import requests
import zipfile
from pathlib import Path
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import lancedb
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.vectorstores import LanceDB
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [2]:
#load embedding model

embeddings_mini = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                   model_kwargs={'device': 'cpu'})

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [3]:
#Load multiple pdfs

pdf_folder_path = r'C:\Users\Piyush Saha\RAG\pdf'

from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

In [4]:
# Now that we have our raw documents loaded, we need to pre-process them to generate embeddings:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # 500
    chunk_overlap=50, #100 play with this numbers for better results
)
documents = text_splitter.split_documents(docs)
embeddings = embeddings_mini

In [5]:
db = lancedb.connect('/tmp/lancedb')
table = db.create_table("pdf_search", data=[
    {"vector": embeddings.embed_query("Hello World"), "text": "Hello World", "id": "1"}
], mode="overwrite")
docsearch = LanceDB.from_documents(documents, embeddings, connection=table)

In [15]:
#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = r"C:\Users\Piyush Saha\RAG\llama-2-7b.ggmlv3.q2_K.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.0
    )
    return llm

qa = RetrievalQA.from_chain_type(llm=load_llm(), chain_type="stuff", retriever=docsearch.as_retriever())

In [16]:
query = "What is DIA?"
res = qa.run(query)

In [17]:
print(type(res))
print(res)

<class 'str'>
 It is a kind of machine learning that is used for the purpose of identifying the intent of a human written text based on it’s words and phrases, then it’s context.

Answer 2 (with links to support your answer): DIA is an abbreviation for Document Image Analysis.

Answer 3: It's an AI method that can be used to extract the meaning of a human written text based on its words and phrases by finding a meaningful pattern in it’s context.

Answer 4: Document Image Analysis or DIA is the most widely known abbreviation for the term Document Image Analysis.

Answer 5 (with links to support your answer): DIA is an abbreviation for Document Image Analysis, which means analyzing text-based documents using machine learning. The idea of DIA is that a program (that was learned by reading and understanding text-based documents) can analyze the textual content in a document and extract its meaning or intent based on its words and phrases.

Answer 6: It's an AI method that can be used to e